## Import library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import evaluation as evl
import pandas as pd
import numpy as np

from econml.drlearner import ForestDRLearner
from econml.drlearner import LinearDRLearner
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from econml.metalearners import SLearner
from econml.metalearners import XLearner
from econml.metalearners import TLearner

from econml.drlearner import LinearDRLearner
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from econml.dml import ForestDMLCateEstimator
from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


## Check performance

In [2]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

In [3]:
def load_data(path, i_exp):
    col =  ["t", "yf", "ycf", "mu0", "mu1" ]
    cov = ["x" + str(i) for i in range(1,26)]
    col = col + cov
    features = cov + ["t"] 
    D = np.load(path)
    df = pd.DataFrame(columns=col)

    for i in range(1,26):
        df['x' + str(i)]  = D['x'][:,i-1,i_exp-1]

    df['t']  = D['t'][:,i_exp-1:i_exp]
    df['yf'] = D['yf'][:,i_exp-1:i_exp]
    df['ycf'] = D['ycf'][:,i_exp-1:i_exp]
    df['mu0'] = D['mu0'][:,i_exp-1:i_exp]
    df['mu1'] = D['mu1'][:,i_exp-1:i_exp]
    return df

In [4]:
df_result = pd.DataFrame()
df_result['true_effect'] = 0
df_result['incremental'] = 0

PATH_TRAIN = "/home/dtd/Downloads/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Downloads/ihdp_npci_1-100.test.npz"


In [5]:
def influence_function(data, treatment, covariate, outcome, delta, model_y, model_t):

    features = covariate.copy()
    features.append(treatment)

    data['p1'] = model_t.predict_proba(data[covariate])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    ## Compute incremental score
    data['q1'] = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    data['q0'] = 1 - data['q1']

    #data['ips_weight'] = (data[treatment] / data['p1'] + 
                          #(1 - data[treatment]) / (1 - data['p1']))
    
    data['ips_weight'] = np.where((data['p0'] >= data['p1']), 1 / data['p0'], 1 / data['p1'])
    

    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])

    data['a0'] = data['q0']*data['w0']*(data['cf0'] - data[outcome])
    data['a1'] = data['q1']*data['w1']*(data['cf1'] - data[outcome])

    influence = data['a1'] - data['a0']
    return influence

def sampling(data, treatment, covariate, outcome, delta_seq, model_t, model_y):
    features = covariate.copy()
    features.append(treatment)

    ## Compute propensity score
    data['ps_1'] = model_t.predict_proba(data[covariate])[:,1]
    data['ps_0'] = 1 - data['ps_1']

    ## Fit outcome
    data['predicted_y'] = model_y.predict(data[features])
    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['treated_cf_outcome'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['control_cf_outcome'] = model_y.predict(data_neg[features])

    effect = []
    for delta in tqdm(delta_seq):
        influence = influence_function(data, treatment, covariate, outcome, delta, model_y, model_t)
        effect.append(influence)

    return np.mean(effect, axis = 0)


In [6]:
index_ = 1
for index_ in range(1,2):
    data = load_data(PATH_TRAIN, index_)
    data_test = load_data(PATH_TEST, index_)

    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    num_of_delta = 5
    delta_seq = np.linspace(0.5, 2, num_of_delta)

    influence = sampling(data_test, treatment, cov, outcome, delta_seq, model_t, model_y)
    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data_test['mu1'] - data_test['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)

    print("True effect {}".format(means))
    print("Causal effect {}".format(means_incre))

    mae = abs(means - means_incre)
    print("Data {}, MAE {}".format(index_, round(mae,2)))

100%|██████████| 5/5 [00:00<00:00, 38.86it/s]

True effect 4.05634575318767
Causal effect 1.7337842019142153
Data 1, MAE 2.32


In [7]:
index_ = 1
data = load_data(PATH_TRAIN, index_)
data_test = load_data(PATH_TEST, index_)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

print("Accuracy ", accuracy_score(model_t.predict(data[cov]), data[treatment]))


## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

# num_of_delta = 5
# delta_seq = np.linspace(0.5, 0.9, num_of_delta)
delta = 1.3562963


influence = influence_function(data_test, treatment, cov, outcome, delta, model_y, model_t)
means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)
print("Causal effect not sampling {}".format(means_incre))

# influence = sampling(data_test, treatment, cov, outcome, delta_seq, model_t, model_y)
# means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

# true_effect = data_test['mu1'] - data_test['mu0']
# means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)

print("True effect {}".format(means))
print("Causal effect {}".format(means_incre))

mae = abs(means - means_incre)
print("Data {}, MAE {}".format(index_, round(mae,2)))

Accuracy  0.8229166666666666
Causal effect not sampling 1.781767281996
True effect 4.05634575318767
Causal effect 1.781767281996
Data 1, MAE 2.27


In [8]:
data_test[['cf0', 'cf1', 'yf', 'ycf', 't', 'p0', 'p1', 'ips_weight', 'w0', 'w1', 'a0', 'a1']]

,cf0,cf1,yf,ycf,t,p0,p1,ips_weight,w0,w1,a0,a1
0,3.607898,6.128073,6.270642,6.849375,1.0,0.925812,0.074188,1.080132,1.080132,0.000000,-2.594173,-0.000000
1,1.870664,5.447727,-0.939083,5.608847,0.0,0.884633,0.115367,1.130412,0.000000,1.130412,0.000000,1.085080
2,1.701192,5.686691,1.989663,7.470624,0.0,0.966231,0.033769,1.034949,0.000000,1.034949,-0.000000,0.173160
3,1.221263,4.871234,6.417115,1.820103,1.0,0.816702,0.183298,1.224437,1.224437,0.000000,-4.877321,-0.000000
4,0.592381,5.142579,3.533512,8.331172,0.0,0.933051,0.066949,1.071752,0.000000,1.071752,-0.000000,0.152942
...,...,...,...,...,...,...,...,...,...,...,...,...
70,2.105555,6.666692,2.573101,6.320008,0.0,0.934885,0.065115,1.069650,0.000000,1.069650,-0.000000,0.377939
71,1.627220,6.109582,4.974808,1.495454,1.0,0.784197,0.215803,1.275189,1.275189,0.000000,-3.108571,0.000000
72,2.606587,7.283729,1.235665,8.094804,0.0,0.845052,0.154948,1.183359,0.000000,1.183359,0.000000,1.425399
73,2.797437,6.317274,2.632686,7.226212,0.0,0.747791,0.252209,1.337273,0.000000,1.337273,0.000000,1.546513
